In [10]:
import pandas as pd
import spacy

# adjusting display options
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('../Dataset/sample_submission.csv')
n_rows = df.shape[0]
df

,id,text
0,0,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge
1,1,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and
2,2,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice
3,3,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap
4,4,hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle
5,5,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle


In [11]:
import spacy

# Load spaCy's English language model
nlp = spacy.load('en_core_web_sm')

# Create vocabulary (reuse from your training phase)
tokenized_lines = [["example", "tokens"], ["for", "vocabulary", "creation"]]
vocab = {word: i for i, word in enumerate(set(word for tokens in tokenized_lines for word in tokens))}
vocab['unk'] = len(vocab)  # Adding 'unk' for unknown tokens

# Display the updated vocabulary size for verification
print("Vocabulary Size:", len(vocab))

Vocabulary Size: 6


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Calculate max_len based on the training data (example value, adjust as necessary)
max_len = 100  # Adjust this value based on your actual data

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(vocab), output_dim=128, input_length=max_len))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))  # Ensure this returns sequences
model.add(Dense(len(vocab), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model summary
model.summary()

C:\Users\mebub_9a7jdi8\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Prepare sequence for prediction
def prepare_sequence(text, vocab):
    tokens = text.lower().split()
    indices = [vocab.get(token, vocab['unk']) for token in tokens]  # Using 'unk' for unknown tokens
    return indices

# Decode predictions
def decode_predictions(preds, vocab):
    inv_vocab = {v: k for k, v in vocab.items()}
    decoded_sequences = []
    for pred in preds:
        decoded_sequence = []
        for word_probs in pred:
            decoded_sequence.append(inv_vocab[np.argmax(word_probs)])
        decoded_sequences.append(decoded_sequence)
    return decoded_sequences

# Prepare each row, pad and predict
predicted_texts = []
for text in df['text']:
    sequence = prepare_sequence(text, vocab)
    padded_sequence = pad_sequences([sequence], maxlen=max_len, padding='post')
    prediction = model.predict(padded_sequence)
    decoded_sequence = decode_predictions(prediction, vocab)[0]
    predicted_texts.append(' '.join(decoded_sequence))

# Display predicted texts for verification
print("Predicted Texts:\n", predicted_texts)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 655ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted Texts:
 ['vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary creation creation creation creation creation creation creation creation creation creation creation creation creation creation creation creation creation creation creation vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabulary vocabula

In [14]:
# Add predictions to the DataFrame
df['predicted_text'] = predicted_texts

# Save to CSV
df[['id', 'predicted_text']].to_csv('../Submission File/submission.csv', index=False)

print("Submission file created successfully!")

Submission file created successfully!
